# Nettoyage du fichier de contact presse 

In [1]:
import re
import pandas as pd
from validate_email import validate_email

In [2]:
# Choix de la méthode de validation (1 ou 2)
valid_method = 2

In [3]:
# Chargement du fichier CSV contenant les adresses e-mail
df = pd.read_csv('data/Contacts_clean.csv')
print(df.shape)
df.head()

(7450, 2)


,Nom organisation,Email contact
0,UPRIGHT AGENCY,ivictor@lOOpourlOOgolf.com
1,UPRIGHT AGENCY,ivictor@lOOpourlOOgolf.com
2,VOCATIF,vocat1f@wanadoo.fr
3,VOCATIF,vocat1f@wanadoo.fr
4,WAKE UP EVENTS,conlacl@scapevenl.com


In [4]:
# Retrait des doublons
df = df.drop_duplicates()
print(df.shape)
df.head()

(3718, 2)


,Nom organisation,Email contact
0,UPRIGHT AGENCY,ivictor@lOOpourlOOgolf.com
2,VOCATIF,vocat1f@wanadoo.fr
4,WAKE UP EVENTS,conlacl@scapevenl.com
6,WHAT'S HAP,maumus@whatshap.fr
8,WMH PROJECT,developpement@fc2events.fr


In [5]:
# Validation des adresses e-mail
def is_valid_email(email):
    is_valid = validate_email(
    email_address=email,
    check_format=True,
    check_blacklist=True,
    check_dns=False,
    dns_timeout=10,
    check_smtp=True,
    smtp_timeout=10,
    smtp_helo_host=False,
    smtp_from_address=False,
    smtp_skip_tls=True,
    smtp_tls_context=False,
    smtp_debug=False)
    
    return is_valid

In [6]:
# Dictionnaires d'erreurs courantes d'OCR
ocr_corrections = {
    # Chiffres confondus avec des lettres
    '1': 'i',  # 1 -> i (sauf dans les chiffres légitimes)
    '0': 'o',  # 0 -> o dans certains contextes
    '5': 's',  # 5 -> s
    '10': 'io',  # 10 -> io
    '01': 'oi',  # 10 -> io
    # Autres corrections possibles
    'rn': 'm',  # rn -> m
    'vv': 'w',  # vv -> w
    'conlacl': 'contact',  # conlacl -> contact
    'redacl0n': 'redaction',  # redacl0n -> redaction
    "cenlr": "centr",  # cen1r -> centr
    'cen1r': 'centr',  # cen1r -> centr
    'conlact': 'contact',  # conlact -> contact
    "redaclion": "redaction",  # redacl10n -> redaction
    "commun%2Ccatlon": "communication",  # commun%2Ccatlon -> communication
}

In [7]:
# Corrections courantes d'OCR
def correct_ocr_errors_advanced(email, corrections_dict=ocr_corrections):
    """Corrige les erreurs OCR courantes dans les emails"""
    if not isinstance(email, str):
        return str(email) if email is not None else ""
    
    email = email.strip()
    
    if not email or '@' not in email:
        return email
    
    
    # Séparer la partie locale et le domaine
    local, domain = email.rsplit('@', 1)
    
    # Appliquer les corrections sur la partie locale et le domaine
    for wrong, correct in corrections_dict.items():
        # Correction pour les caractères simples (1, 0, 5)
        if len(wrong) == 1 and len(correct) == 1:
            # Remplacer uniquement si entouré de lettres (pas de chiffres)
            local = re.sub(rf'(?<=[a-zA-Z]){re.escape(wrong)}(?=[a-zA-Z])', correct, local)
            local = re.sub(rf'^{re.escape(wrong)}(?=[a-zA-Z])', correct, local)
            local = re.sub(rf'(?<=[a-zA-Z]){re.escape(wrong)}$', correct, local)
            
            domain = re.sub(rf'(?<=[a-zA-Z]){re.escape(wrong)}(?=[a-zA-Z])', correct, domain)
            domain = re.sub(rf'^{re.escape(wrong)}(?=[a-zA-Z])', correct, domain)
            domain = re.sub(rf'(?<=[a-zA-Z]){re.escape(wrong)}$', correct, domain)
        else:
            # Correction pour les groupes de caractères (rn, vv)
            local = local.replace(wrong, correct)
            domain = domain.replace(wrong, correct)
    
    email_corrected = f"{local}@{domain}"
    
    return email_corrected

In [8]:
# Correction des mails
df['corrected_email'] = df["Email contact"].apply(lambda email: correct_ocr_errors_advanced(email))
print(df.shape)
df.head()

(3718, 3)


,Nom organisation,Email contact,corrected_email
0,UPRIGHT AGENCY,ivictor@lOOpourlOOgolf.com,ivictor@lOOpourlOOgolf.com
2,VOCATIF,vocat1f@wanadoo.fr,vocatif@wanadoo.fr
4,WAKE UP EVENTS,conlacl@scapevenl.com,contact@scapevenl.com
6,WHAT'S HAP,maumus@whatshap.fr,maumus@whatshap.fr
8,WMH PROJECT,developpement@fc2events.fr,developpement@fc2events.fr


In [9]:
# Validation complexe
if valid_method == 1:

    # Test sur un échatillon
    df_test = df.sample(n=20, random_state=42)

    # Validation des adresses e-mail
    df_test["Valid_Email"] = df_test["Email contact"].apply(is_valid_email)
    print(df_test.shape)

    # Résumé
    df_test["Valid_Email"].value_counts()

    # Affichage
    print(df_test)

In [10]:
# Autre méthode de validation
from email_validator import validate_email, EmailNotValidError
def is_valid_email_v2(email):
    try:
        # Validate.
        v =validate_email(email, check_deliverability=True)
        # Email is valid.
        is_valid = True
        proposed_email = v["email"]
    except EmailNotValidError as e:
        # Email is not valid.
        is_valid = False
        proposed_email = None

    return pd.Series([is_valid, proposed_email])

In [11]:
# Validation plus rapide 
if valid_method == 2:
    df[['Valid_Email', 'proposed_email']] = df.apply(lambda row: is_valid_email_v2(row["corrected_email"]), axis=1)
    print(df.shape)
    df.head()

C:\Users\aurel\AppData\Local\Temp\ipykernel_33448\2326700017.py:9: DeprecationWarning: dict-like access to the return value of validate_email is deprecated and may not be supported in the future.
  proposed_email = v["email"]


(3718, 5)


In [12]:
# Résumé
df["Valid_Email"].value_counts()

Valid_Email
True     2567
False    1151
Name: count, dtype: int64

In [13]:
# Nombre d'organisations uniques avec au moins 1 email valide
unique_organizations = df.groupby("Valid_Email")["Nom organisation"].nunique()
unique_organizations

Valid_Email
False     852
True     1581
Name: Nom organisation, dtype: int64

In [14]:
# Nombre de contacts a priori valide par organisation
df[df["Valid_Email"]].groupby("Nom organisation").nunique()

,Email contact,corrected_email,Valid_Email,proposed_email
Nom organisation,,,,
30,3,3,1,3
ET,1,1,1,1
Miroir de l'Arl C9 Ier de chaque mois Mensuel,1,1,1,1
!FESTIVAL,1,1,1,1
0,170,168,1,168
...,...,...,...,...
tu,1,1,1,1
tü,22,22,1,22
z,2,2,1,2
